In [28]:
#imports
import csv
import requests
import json
import os      # for files
import csv     # csv file handling
import codecs  # because text file encoding is an issue in 2018
from datetime import datetime, date #time

In [29]:
competitionID = 119 # AT Bundesliga
seasonID = 2018     #2018/2019
requestURLMatchStatic = F"http://omo.akamai.opta.net/competition.php?competition={competitionID}&season_id={seasonID}&feed_type=F1&user=Hackweekend&psw=35BJ10nwf&json"
r = requests.get(requestURLMatchStatic
                         ,timeout=None
                         ,headers={  'Content-Type': 'application/json'}
                        )    
jsondata = r.json()
docMatch = jsondata['SoccerFeed']['SoccerDocument']['MatchData']
#get all matches
matchList = []
for match in docMatch:
    matchObj = match["@attributes"]
#    print(matchObj)
    matchID_long = match["@attributes"]["uID"]
    matchID = matchID_long.replace('g','')
    matchList.append(matchID)
print(matchList)

['988749', '988751', '988750', '988753', '988752', '988754', '988757', '988755', '988756', '988759', '988758', '988760', '988762', '988761', '988766', '988765', '988764', '988763', '988767', '988772', '988769', '988771', '988770', '988768', '988773', '988774', '988775', '988777', '988776', '988778', '988780', '988781', '988779', '988782', '988784', '988783', '988785', '988787', '988786', '988789', '988790', '988788', '988794', '988792', '988795', '988791', '988796', '988793', '988799', '988801', '988797', '988798', '988800', '988802', '988805', '988806', '988807', '988808', '988803', '988804', '988809', '988810', '988814', '988811', '988813', '988812', '988819', '988817', '988820', '988816', '988818', '988815', '988823', '988822', '988826', '988825', '988821', '988824', '988832', '988828', '988827', '988830', '988831', '988829', '988834', '988836', '988838', '988835', '988833', '988837', '988843', '988841', '988842', '988839', '988844', '988840', '988846', '988848', '988849', '988845',

In [30]:
#get all matches with winners
u=1
allrows=[]
print("trying to import matches: "+str(len(matchList)))
while u<len(matchList):    
    gameID=matchList[u]
    print("Parsing GameID: "+str(gameID))
    u=u+1
    r = requests.get(F"http://omo.akamai.opta.net/?game_id={gameID}&feed_type=F9&user=Hackweekend&psw=35BJ10nwf&json"
                         ,timeout=None
                         ,headers={  'Content-Type': 'application/json'}
                        )    
    jsondata = r.json()
    if 'SoccerFeed' in jsondata:
        matchData = jsondata['SoccerFeed']['SoccerDocument']['MatchData']
        teamdata = matchData['TeamData']
        onerow = {}
        onerow['matchid']=gameID;
        onerow['matchdata']=matchData['MatchInfo']['Date']
        potwinner=matchData['MatchInfo']['Result']['@attributes']
        if 'Winner' not in potwinner:
            onerowwinner='none'
        else:
            onerowwinner = matchData['MatchInfo']['Result']['@attributes']['Winner']
        teamRefs = []
        #statreflist for one time setting
        statreflist= [];
        i=0
        while i<len(teamdata):
            teamstats = teamdata[i] 
            onerow['team_'+str(i)+'_teamRef']= teamstats['@attributes']['TeamRef']
            teamstatslist = teamstats['Stat']
            teamRefs.append(teamstats['@attributes']['TeamRef'])
            if len(statreflist)==0:
                l=0
                while l<len(teamstatslist):
                    stat = teamstatslist[l]
                    stattype = stat['@attributes']['Type']
                    #TODO what up with fh and sh separation?            
                    statvalue = stat['@value']
                    onerow['team_'+str(i)+'_s_'+stattype] = float(statvalue)
                    statreflist.append(stattype);
                    l=l+1
            else :
                usedstats=[]
                l=0
                while l<len(teamstatslist):
                    stat = teamstatslist[l]
                    stattype = stat['@attributes']['Type']
                    statvalue = stat['@value']
                    if stattype in statreflist:
                        usedstats.append(stattype)
                        onerow['team_'+str(i)+'_s_'+stattype] = float(statvalue)
                    l=l+1
                #anything that is in statreflist and not in usedstats needs value 0   
                openstats = list(set(statreflist).difference(set(usedstats)))
                if len(openstats)>0:
                    k=0;
                    while k<len(openstats):
                        onerow['team_'+str(i)+'_s_'+openstats[k]] = 0
                        k=k+1
            i=i+1
        #who is winner team_0 or team_1
        z=0
        while z<len(teamRefs):
            if teamRefs[z]==onerowwinner:
                onerow['winner']='team_'+str(z)
            z=z+1
        if 'winner' not in onerow:
            onerow['winner']='draw'
        allrows.append(onerow)

trying to import matches: 132
Parsing GameID: 988751
Parsing GameID: 988750
Parsing GameID: 988753
Parsing GameID: 988752
Parsing GameID: 988754
Parsing GameID: 988757
Parsing GameID: 988755
Parsing GameID: 988756
Parsing GameID: 988759
Parsing GameID: 988758
Parsing GameID: 988760
Parsing GameID: 988762
Parsing GameID: 988761
Parsing GameID: 988766
Parsing GameID: 988765
Parsing GameID: 988764
Parsing GameID: 988763
Parsing GameID: 988767
Parsing GameID: 988772
Parsing GameID: 988769
Parsing GameID: 988771
Parsing GameID: 988770
Parsing GameID: 988768
Parsing GameID: 988773
Parsing GameID: 988774
Parsing GameID: 988775
Parsing GameID: 988777
Parsing GameID: 988776
Parsing GameID: 988778
Parsing GameID: 988780
Parsing GameID: 988781
Parsing GameID: 988779
Parsing GameID: 988782
Parsing GameID: 988784
Parsing GameID: 988783
Parsing GameID: 988785
Parsing GameID: 988787
Parsing GameID: 988786
Parsing GameID: 988789
Parsing GameID: 988790
Parsing GameID: 988788
Parsing GameID: 988794
Pars

In [31]:
print("FOUND: "+str(len(allrows)))

FOUND: 83


In [32]:
import pandas as pd
allMatchData = pd.DataFrame.from_dict(allrows)
allMatchData.fillna(0, inplace=True)

In [33]:
allMatchData.head()

,matchdata,matchid,team_0_s_accurate_back_zone_pass,team_0_s_accurate_chipped_pass,team_0_s_accurate_corners_intobox,team_0_s_accurate_cross,team_0_s_accurate_cross_nocorner,team_0_s_accurate_flick_on,team_0_s_accurate_freekick_cross,team_0_s_accurate_fwd_zone_pass,...,team_1_s_total_throws,team_1_s_total_yel_card,team_1_s_touches,team_1_s_touches_in_opp_box,team_1_s_unsuccessful_touch,team_1_s_won_contest,team_1_s_won_corners,team_1_s_won_tackle,team_1_teamRef,winner
0,20180728T160000+0100,988751,140.0,17.0,1.0,4.0,3.0,0.0,0.0,151.0,...,18.0,1.0,530.0,24.0,15.0,6.0,9.0,15.0,t2089,team_1
1,20180728T160000+0100,988750,225.0,30.0,0.0,1.0,1.0,1.0,0.0,166.0,...,16.0,1.0,431.0,9.0,8.0,5.0,6.0,12.0,t4950,team_0
2,20180729T160000+0100,988753,114.0,6.0,1.0,2.0,1.0,2.0,0.0,60.0,...,27.0,2.0,928.0,24.0,14.0,6.0,5.0,13.0,t212,team_1
3,20180729T160000+0100,988752,207.0,13.0,0.0,5.0,5.0,2.0,0.0,234.0,...,24.0,4.0,445.0,10.0,12.0,4.0,4.0,12.0,t280,team_0
4,20180729T160000+0100,988754,72.0,12.0,5.0,9.0,4.0,3.0,0.0,128.0,...,32.0,2.0,619.0,26.0,13.0,5.0,3.0,10.0,t5666,team_0


In [34]:
list(allMatchData)

['matchdata',
 'matchid',
 'team_0_s_accurate_back_zone_pass',
 'team_0_s_accurate_chipped_pass',
 'team_0_s_accurate_corners_intobox',
 'team_0_s_accurate_cross',
 'team_0_s_accurate_cross_nocorner',
 'team_0_s_accurate_flick_on',
 'team_0_s_accurate_freekick_cross',
 'team_0_s_accurate_fwd_zone_pass',
 'team_0_s_accurate_goal_kicks',
 'team_0_s_accurate_keeper_sweeper',
 'team_0_s_accurate_keeper_throws',
 'team_0_s_accurate_launches',
 'team_0_s_accurate_layoffs',
 'team_0_s_accurate_long_balls',
 'team_0_s_accurate_pass',
 'team_0_s_accurate_pull_back',
 'team_0_s_accurate_through_ball',
 'team_0_s_accurate_throws',
 'team_0_s_aerial_lost',
 'team_0_s_aerial_won',
 'team_0_s_att_assist_openplay',
 'team_0_s_att_assist_setplay',
 'team_0_s_att_bx_centre',
 'team_0_s_att_bx_left',
 'team_0_s_att_bx_right',
 'team_0_s_att_cmiss_high',
 'team_0_s_att_cmiss_high_left',
 'team_0_s_att_cmiss_high_right',
 'team_0_s_att_cmiss_left',
 'team_0_s_att_cmiss_right',
 'team_0_s_att_fastbreak',
 

In [35]:
allMatchData.describe()

,team_0_s_accurate_back_zone_pass,team_0_s_accurate_chipped_pass,team_0_s_accurate_corners_intobox,team_0_s_accurate_cross,team_0_s_accurate_cross_nocorner,team_0_s_accurate_flick_on,team_0_s_accurate_freekick_cross,team_0_s_accurate_fwd_zone_pass,team_0_s_accurate_goal_kicks,team_0_s_accurate_keeper_sweeper,...,team_1_s_total_tackle,team_1_s_total_through_ball,team_1_s_total_throws,team_1_s_total_yel_card,team_1_s_touches,team_1_s_touches_in_opp_box,team_1_s_unsuccessful_touch,team_1_s_won_contest,team_1_s_won_corners,team_1_s_won_tackle
count,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,...,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000
mean,156.265060,18.566265,1.650602,4.590361,2.951807,1.626506,0.349398,159.469880,4.578313,0.253012,...,17.048193,0.204819,25.180723,1.662651,595.036145,17.168675,14.493976,4.578313,4.951807,12.313253
std,53.424915,7.447692,1.310685,2.930230,2.336892,1.470945,0.705963,58.736148,2.409941,0.489982,...,3.941349,0.579298,6.878997,1.363971,110.417439,8.720342,4.058636,3.084661,2.958673,3.428459
min,60.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,51.000000,0.000000,0.000000,...,6.000000,0.000000,12.000000,0.000000,415.000000,2.000000,6.000000,0.000000,0.000000,4.000000
25%,115.000000,13.000000,1.000000,3.000000,1.000000,1.000000,0.000000,118.500000,3.000000,0.000000,...,14.000000,0.000000,21.000000,1.000000,505.000000,10.500000,12.000000,2.500000,3.000000,10.000000
50%,154.000000,18.000000,1.000000,4.000000,3.000000,1.000000,0.000000,151.000000,4.000000,0.000000,...,17.000000,0.000000,24.000000,1.000000,577.000000,15.000000,14.000000,4.000000,4.000000,12.000000
75%,200.500000,24.000000,2.000000,6.000000,4.000000,2.000000,1.000000,193.500000,6.000000,0.000000,...,20.000000,0.000000,29.000000,3.000000,673.500000,22.500000,17.000000,6.000000,7.000000,14.000000
max,272.000000,38.000000,6.000000,13.000000,10.000000,7.000000,4.000000,335.000000,13.000000,2.000000,...,27.000000,3.000000,47.000000,6.000000,928.000000,38.000000,26.000000,18.000000,14.000000,23.000000


In [36]:
allMatchData.dtypes

matchdata                             object
matchid                               object
team_0_s_accurate_back_zone_pass     float64
team_0_s_accurate_chipped_pass       float64
team_0_s_accurate_corners_intobox    float64
team_0_s_accurate_cross              float64
team_0_s_accurate_cross_nocorner     float64
team_0_s_accurate_flick_on           float64
team_0_s_accurate_freekick_cross     float64
team_0_s_accurate_fwd_zone_pass      float64
team_0_s_accurate_goal_kicks         float64
team_0_s_accurate_keeper_sweeper     float64
team_0_s_accurate_keeper_throws      float64
team_0_s_accurate_launches           float64
team_0_s_accurate_layoffs            float64
team_0_s_accurate_long_balls         float64
team_0_s_accurate_pass               float64
team_0_s_accurate_pull_back          float64
team_0_s_accurate_through_ball       float64
team_0_s_accurate_throws             float64
team_0_s_aerial_lost                 float64
team_0_s_aerial_won                  float64
team_0_s_a

# Parse all Events with matchid

## Coding Table preparation

In [67]:
import os 
cwd = os.getcwd()
print(cwd)
import_file = "./../db/qualifiers.csv" 
#ID;Qualifier;Values;Description
qualifiers_dict = {}
with codecs.open(import_file, 'rU','cp1252') as handle:
    reader = csv.DictReader(handle, delimiter=";", quotechar='"')
    for line in reader:
        qualifiers_dict[line["ID"]] = {'Qualifier' : line["Qualifier"], 'Values' : line['Values'],'Description' : line['Description']}
        
import_file = "./../db/events.csv" 
#Event ID;Name;Description
events_dict = {}
with codecs.open(import_file, 'rU','cp1252') as handle:
    reader = csv.DictReader(handle, delimiter=";", quotechar='"')
    for line in reader:
        events_dict[line["Event ID"]] = {'Name' : line['Name'], 'Description' : line['Description']}
        
#print(qualifiers_dict)
#print(qualifiers_dict['7']) # Players caught offside
#print(events_dict['50'])    # Dispossessed

competitionID = 119 # AT Bundesliga
seasonID = 2018     #2018/2019
requestURLSquad = F"http://omo.akamai.opta.net/competition.php?competition={competitionID}&season_id={seasonID}&feed_type=F40&user=Hackweekend&psw=35BJ10nwf&json"
r = requests.get(requestURLSquad
                         ,timeout=None
                         ,headers={  'Content-Type': 'application/json'}
                        )    
jsondata = r.json()
docSquad = jsondata['SoccerFeed']['SoccerDocument']['Team']
matchSquad = docSquad[1]
#print(json.dumps(matchSquad, indent=4, sort_keys=True))
playerDict = {}
for team in docSquad:
    teamObj = team["@attributes"]
    teamName = teamObj['short_club_name']
    teamID = teamObj['uID'].replace('t','')
    for player in team["Player"]:
        playerID_long = player["@attributes"]["uID"]
        playerID = playerID_long.replace('p','')
        playerName = player["Name"]
        playerDict[playerID] = {'Name' : playerName, 'Team' : teamName, 'TeamID' : teamID}
#print(playerDict)

/home/jovyan/work/rapidhackathon/datasciencelab


# Get All Events for all Matches in Our Match DataFram

In [44]:
parsedMatches = list(allMatchData['matchid'])

interested in successfull events per eventtype per team 

so we need to know for the event the team and if it is a success (if it is team_0 or team 1

In [61]:
gameid='988751'
refRow = allMatchData[allMatchData['matchid'] == gameid]
refValues = refRow[['matchid','winner','team_1_teamRef','team_0_teamRef']]

In [62]:
refValues

,matchid,winner,team_1_teamRef,team_0_teamRef
0,988751,team_1,t2089,t2081


In [141]:
i = 0
events=[]
while i<len(parsedMatches):
    gameID = parsedMatches[i]
    print("parsing Game: "+str(gameID))
    requestURL = F"http://omo.akamai.opta.net/?game_id={gameID}&feed_type=F24&user=Hackweekend&psw=35BJ10nwf&json"
    r = requests.get(requestURL
                         ,timeout=None
                         ,headers={  'Content-Type': 'application/json'}
                        )    
    jsondata = r.json()
    game = jsondata['Games']['Game']    
    gamedata = game['@attributes']['game_date']
    doc = game['Event']
    #get teamRef and stats for this match
    refRow = allMatchData[allMatchData['matchid'] == gameID]
    refValues = refRow[['matchid','winner','team_1_teamRef','team_0_teamRef']]
    #print(refValues)
    #now we know the teamrefs it concerns the part
    #what do we want for each event?
    #per each eventtype the sum of all events before that where sucessfull for 
    #each team?
    #so we make a ref table - which has:
    #team -> team0 or team1 - teamid , eventtype, successfull    
    for event in doc:
    ## direct attributes
        eventline = {}
        eventAttr = event['@attributes']
        outcome = int(eventAttr['outcome'])
        eventline['matchid'] = gameID
        eventline['timestamp'] = eventAttr['timestamp']
        eventline['id'] = eventAttr['id']
        eventline['outcome'] = int(eventAttr['outcome'])
        eventline['type'] = eventAttr['type_id']
        eventline['winner'] = refValues['winner'].values[0]
        # player from ID (if involved)
        playerInvolved = False
        playerName = ""
        try:
            playerID = eventAttr['player_id']
            playerObj = playerDict[playerID]
            playerName = playerObj['Name']
            teamID = "t"+playerObj['TeamID']
            teamName = playerObj['Team']
            playerInvolved = True
        except KeyError:
            playerName = 'NO PLAYER'
            playerInvolved = False
        if playerInvolved:
            team = ''
            isTeam1 = refValues[refValues['team_1_teamRef']==teamID];
            if len(isTeam1)>0:
                team='team_1'
            else: 
                team='team_0'
            eventline['team'] = team
            eventline['teamid'] = teamID
        else : 
            eventline['team'] = 'NONE'
            eventline['teamid'] = 'NONE'
        events.append(eventline)
    i=i+1

parsing Game: 988751
parsing Game: 988750
parsing Game: 988753
parsing Game: 988752
parsing Game: 988754
parsing Game: 988757
parsing Game: 988755
parsing Game: 988756
parsing Game: 988759
parsing Game: 988758
parsing Game: 988760
parsing Game: 988762
parsing Game: 988761
parsing Game: 988766
parsing Game: 988765
parsing Game: 988764
parsing Game: 988763
parsing Game: 988767
parsing Game: 988772
parsing Game: 988769
parsing Game: 988771
parsing Game: 988770
parsing Game: 988768
parsing Game: 988773
parsing Game: 988774
parsing Game: 988775
parsing Game: 988777
parsing Game: 988776
parsing Game: 988778
parsing Game: 988780
parsing Game: 988781
parsing Game: 988779
parsing Game: 988782
parsing Game: 988784
parsing Game: 988783
parsing Game: 988785
parsing Game: 988787
parsing Game: 988786
parsing Game: 988789
parsing Game: 988790
parsing Game: 988788
parsing Game: 988794
parsing Game: 988792
parsing Game: 988795
parsing Game: 988791
parsing Game: 988796
parsing Game: 988793
parsing Game:

In [142]:
allEventsData = pd.DataFrame.from_dict(events)

# Now We have all Events data 
- we need for each event the sum for each eventtype
- for this team 
-> where outcome = 1 and matchid = matchid

In [143]:
len(list(allEventsData.type.unique()))

50

In [144]:
allEventsData['datetest'] = pd.to_datetime(allEventsData['timestamp'])

In [145]:
allEventsData.head()

,id,matchid,outcome,team,teamid,timestamp,type,winner,datetest
0,1543551217,988751,1,NONE,NONE,2018-07-28T15:05:32.59,34,team_1,2018-07-28 15:05:32.590
1,1357993239,988751,1,NONE,NONE,2018-07-28T15:09:55.17,34,team_1,2018-07-28 15:09:55.170
2,1634877849,988751,1,NONE,NONE,2018-07-28T16:00:12.288,32,team_1,2018-07-28 16:00:12.288
3,1097243024,988751,1,NONE,NONE,2018-07-28T16:00:12.288,32,team_1,2018-07-28 16:00:12.288
4,1184172299,988751,1,team_0,t2081,2018-07-28T16:00:13.432,1,team_1,2018-07-28 16:00:13.432


In [146]:
len(allEventsData)

137699

In [147]:
allEventsData.to_csv("allEventsData.csv", encoding="utf-8") 

In [130]:
allMatchData.to_csv("allMatchData.csv", encoding="utf-8") 